In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm

In [20]:
# Ler os dados
train_df = pd.read_csv('datasets/train.csv')
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [21]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [22]:
train_df = train_df.drop(columns='Name')
train_df = train_df.drop(columns='Ticket')
train_df = train_df.drop(columns='Cabin')
train_df = train_df.drop(columns='PassengerId')

In [23]:
# O mesmo pra Fare
f = np.mean(train_df['Fare'])
train_df['Fare'] = train_df['Fare'].replace(np.nan, f)

In [24]:
# Tamanho da familia. Penso que eh um parametro util pois um individuo poderia morrer tentando salvar um mombro da sua familia.
# Sortudo os que viajaram sozinhos? Veremos.
train_df['Family_Size'] = train_df['SibSp'] + train_df['Parch']

In [25]:
# Preco do bilhete, um bilhete mais caro -> suites ou coisas do tipo(?)
train_df['Fare_Per_Person'] = train_df['Fare'] / (train_df["Family_Size"] + 1)

In [26]:
train_df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,Fare_Per_Person
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.904602,19.916375
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,1.613459,35.841257
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000,7.250000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000,8.300000
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000,23.666667
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,10.000000,512.329200


In [27]:
s = 0
q = 0
c = 0
for i in range(len(train_df['Embarked'])):
    if train_df.loc[i, 'Survived'] == 1 and train_df.loc[i, 'Embarked'] == 'S':
        s = s + 1
    elif train_df.loc[i, 'Survived'] == 1 and train_df.loc[i, 'Embarked'] == 'Q':
        q= q + 1
    elif train_df.loc[i, 'Survived'] == 1 and train_df.loc[i, 'Embarked'] == 'C':
        c = c + 1
y = train_df.loc[train_df['Survived'] == 1]
print(s/len(y))
print(q/len(y))
print(c/len(y))

0.6345029239766082
0.08771929824561403
0.2719298245614035


In [28]:
train_df['Embarked'].fillna('Q', inplace=True)

In [29]:
mapeamento = {'S': 1, 'C': 0, 'Q': -1}
train_df["Embarked"] = train_df['Embarked'].replace(mapeamento)
train_df['Embarked']

0      1
1      0
2      1
3      1
4      1
      ..
886    1
887    1
888    1
889    0
890   -1
Name: Embarked, Length: 891, dtype: int64

In [30]:
train_df["Embarked"]

0      1
1      0
2      1
3      1
4      1
      ..
886    1
887    1
888    1
889    0
890   -1
Name: Embarked, Length: 891, dtype: int64

In [31]:
# Substituir a falta de dados sobre a idade com a media das restantes.
k = np.mean(train_df['Age'])
train_df['Age'].fillna(k, inplace=True)
'''for i in range(len(train_df['Age'])):
    if train_df.loc[i, 'Age'] == -1:
        train_df.loc[i, 'Age'] = k * (train_df.loc[i, 'Family_Size'] + 1)'''
train_df['Age']

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [32]:
train_df['Sex'] = train_df['Sex'].replace("female", int(1))
train_df['Sex'] = train_df['Sex'].replace("male", int(0))
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family_Size,Fare_Per_Person
0,0,3,0,22.000000,1,0,7.2500,1,1,3.62500
1,1,1,1,38.000000,1,0,71.2833,0,1,35.64165
2,1,3,1,26.000000,0,0,7.9250,1,0,7.92500
3,1,1,1,35.000000,1,0,53.1000,1,1,26.55000
4,0,3,0,35.000000,0,0,8.0500,1,0,8.05000
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,1,0,13.00000
887,1,1,1,19.000000,0,0,30.0000,1,0,30.00000
888,0,3,1,29.699118,1,2,23.4500,1,3,5.86250
889,1,1,0,26.000000,0,0,30.0000,0,0,30.00000


In [33]:
'''scaler = MinMaxScaler()
train_df[['Age', 'Fare', 'Family_Size']] = scaler.fit_transform(train_df[['Age', 'Fare', 'Family_Size']])
train_df'''

"scaler = MinMaxScaler()\ntrain_df[['Age', 'Fare', 'Family_Size']] = scaler.fit_transform(train_df[['Age', 'Fare', 'Family_Size']])\ntrain_df"

In [45]:
# Avaliar o que podemos usar para treinar um modelo
# O Id eh util? Certamente nao, pois so esta em ordem crescente, assim como o index.
scaler = StandardScaler()
y_train = train_df['Survived']
X_train = scaler.fit_transform(train_df[['Sex', 'Age','Pclass', 'Fare_Per_Person', 'Fare', 'Family_Size']])

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [49]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    print(clf.__class__.__name__, cross_val_score(clf, X_train, y_train, cv=4, scoring='accuracy'))

LogisticRegression [0.78475336 0.81165919 0.77130045 0.7972973 ]
RandomForestClassifier [0.74887892 0.83408072 0.8161435  0.81981982]
SVC [0.8206278  0.83856502 0.79820628 0.83333333]
VotingClassifier [0.8206278  0.85650224 0.79820628 0.83333333]


In [50]:
rf_clf = RandomForestClassifier(random_state=42)
cross_val_score(rf_clf, X_train, y_train, cv=2, scoring='accuracy')

array([0.76008969, 0.78202247])

In [51]:
logit_clf = LogisticRegression(random_state=42)
cross_val_score(logit_clf, X_train, y_train, cv=2, scoring='accuracy')

array([0.78923767, 0.79101124])

In [52]:
k_clf = KNeighborsClassifier()
cross_val_score(k_clf, X_train, y_train, cv=2, scoring='accuracy')

array([0.76457399, 0.82022472])

In [53]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [54]:
test = pd.read_csv('datasets/test.csv')
test = test.drop(columns='Name')
test = test.drop(columns='Ticket')
test = test.drop(columns='Cabin')
test = test.drop(columns='Embarked')
# Substituir a falta de dados sobre a idade com a media das restantes.
k = np.mean(test['Age'])
test['Age'] = test['Age'].replace(np.nan, k)
# O mesmo pra Fare
f = np.mean(test['Fare'])
test['Fare'] = test['Fare'].replace(np.nan, f)
test['Family_Size'] = test['SibSp'] + test['Parch']
test['Fare_Per_Person'] = test['Fare'] / (test["Family_Size"] + 1)
test['Sex'] = test['Sex'].replace("female", int(1))
test['Sex'] = test['Sex'].replace("male", int(0))
X_test = scaler.fit_transform(test[['Sex', 'Age','Pclass', 'Fare_Per_Person', 'Fare', 'Family_Size']])
X_test

array([[-0.75592895,  0.3349926 ,  0.87348191, -0.3939176 , -0.49840706,
        -0.5534426 ],
       [ 1.32287566,  1.32553003,  0.87348191, -0.51565835, -0.51327429,
         0.10564289],
       [-0.75592895,  2.51417495, -0.31581919, -0.34166065, -0.46508846,
        -0.5534426 ],
       ...,
       [-0.75592895,  0.65196458,  0.87348191, -0.4102052 , -0.50879189,
        -0.5534426 ],
       [-0.75592895,  0.        ,  0.87348191, -0.38770852, -0.4944482 ,
        -0.5534426 ],
       [-0.75592895,  0.        ,  0.87348191, -0.40450323, -0.23790598,
         0.76472838]])

In [55]:
y_predict = voting_clf.predict(X_test)

In [60]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [61]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [62]:
d = {'PassengerId' : test['PassengerId'], 'Survived' : y_pred}
out_put = pd.DataFrame(d)

In [63]:
out_put

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [64]:
out_put.to_csv('Titanic_solve.csv', index=False)